In [1]:
import rasterio
import glob
import os,sys
from matplotlib import pyplot as plt
import numpy as np
import fiona
from shapely.geometry import shape
import shapely
from rasterio.mask import mask
from pyproj import Proj, transform

from torch import nn
from torch.nn import functional as F
import torch
from torchvision import models
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import DataLoader

from unet_models import unet11

from test_unet_helpers import calcXYfromRC, checkWindow, gtDatasetSampler2, DigitalGlobeSamplerTensor

from utils import variable
from scipy import misc

from gbdxtools import Interface, CatalogImage




/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/spatial_torch/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expe

In [2]:
# specify scene id for DG data
dg_scene_id = '1030010057062200' # 2016
dg_scene_id = '104001000AA4B500' # 2015
dg_scene_id = '103001001A425F00' #napoli 2011

# specify image for 2.0 meter analysis
gt_image_2 = '../rasters/union_impervious_raster_2_0_0_wgs84.tif' # desktop

# specify images for 0.5 meter analysis
gt_image_05 = '../rasters/union_impervious_raster_0_5.tif' # desktop

# specify the shapefile
shpfile = '../union/union.shp' # desktop

with fiona.open(shpfile) as shp:
    crs = shp.crs
    shp_bounds = shp.bounds

In [3]:
#np.mean(img_2m, axis=(1,2)).compute(), np.std(img_2m, axis=(1,2)).compute()

In [4]:
# img_2m.shape

In [5]:
# connect to gbdx
gbdx = Interface()

# get the dask array for the 8 band MS image
img_2m = CatalogImage(dg_scene_id, band_type='MS', bbox=shp_bounds, acomp=True)
rows, cols = img_2m.shape[1:]

np.mean(img_2m, axis=(1,2)).compute(), np.std(img_2m, axis=(1,2)).compute()

In [6]:
# specify image transform for RGB image
img_transform = Compose([
    Normalize(mean=[1630.7322, 1574.9552, 1549.1031], std=[837.0847 , 766.4114 , 604.51605])
])

# for 2015
img_transform = Compose([
    Normalize(mean=[1281.2312, 1243.6267, 1334.2975], std=[645.607, 703.0832, 862.68665])
])

# Load the model and the model state from the .pt file. It may need to be downloaded from AWS S3

In [7]:
model_path = 'runs/debug/model_e100_b32_no_aug_GPUPAR.pt' # RGB batch size 32
model_path = 'runs/debug/model_e100_b8_no_aug_GPUPAR.pt' # RGB batch size 8
model_path = 'runs/debug/MS_model_e100_b8_no_aug_GPUPAR.pt' # MS model
model_path = 'runs/debug/model_e44_s34938.pt' # RGB early exit (large batch size, i think)

model = unet11(pretrained=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # get device for gpu or cpu
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
    model.to(device)
    
elif torch.cuda.is_available():
    model.to(device)

model.eval()


#load model
if os.path.exists(model_path):
    state = torch.load(str(model_path))
    epoch = state['epoch']
    step = state['step']
    model.load_state_dict(state['model'])
    print('Restored model, epoch {}, step {:,}'.format(epoch, step))

Restored model, epoch 44, step 537


In [8]:
img_2m.shape, torch.cuda.is_available()

((8, 3333, 4579), True)

In [9]:
## try to the full dataset
# input_img = torch.unsqueeze(variable(dg_dataset[test_ind], volatile=True), dim=0)



img_arr = img_2m[[1,2,4], :2048,:2048].compute() #BGR
# img_arr = img_2m[:, :2048,:2048].compute() # all 8
img_arr = img_transform(torch.from_numpy(img_arr))


In [10]:
input_img = torch.unsqueeze(variable(img_arr, volatile=True), dim=0)
del img_arr

/home/ubuntu/segmentation/IS_segmentation/utils.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return cuda(Variable(x, volatile=volatile))


In [11]:
# model.cpu()
# big_mask = model(input_img.cpu())

big_mask = model(input_img)

In [12]:
big_mask.cpu().detach().numpy().squeeze().shape

(2048, 2048)

In [13]:
## write out with gdal
import gdal, osr

def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(4326)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()
    
rname = 'runs/debug/test_eval_BGR_2015_renorm.tif'
aff = img_2m.affine
raster_origin = (aff.c, aff.f)
pixel_height = aff.e
pixel_width = aff.a
array2raster(rname, raster_origin, pixel_width, pixel_height, big_mask.cpu().detach().numpy().squeeze())